In [ ]:
!pip install spektral

     |████████████████████████████████| 123 kB 5.3 MB/s 


In [ ]:
import tensorflow as tf 
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import spektral

In [ ]:
(training_dataset,validation_dataset,testing_dataset) = tfds.load('eurosat',split=['train[:75%]','train[75%:90%]','train[90%:]'],shuffle_files=True,as_supervised=True)

print(len(training_dataset),len(validation_dataset),len(testing_dataset))

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/eurosat/rgb/2.0.0.incompleteZ2AE9Q/eurosat-train.tfrecord


  0%|          | 0/27000 [00:00<?, ? examples/s]

Dataset eurosat downloaded and prepared to /root/tensorflow_datasets/eurosat/rgb/2.0.0. Subsequent calls will reuse this data.
20250 4050 2700


In [ ]:
hyperparameters = {'epochs':200,'lr':1e-3,'optimiser':'adam','num_episodes':100,'batch_size':32}
print(hyperparameters)

{'epochs': 200, 'lr': 0.001, 'optimiser': 'adam', 'num_episodes': 100, 'batch_size': 32}


In [ ]:
training_ds = training_dataset.shuffle(1024).batch(hyperparameters['batch_size']).prefetch(tf.data.AUTOTUNE)
validation_ds = validation_dataset.shuffle(1024).batch(hyperparameters['batch_size']).prefetch(tf.data.AUTOTUNE)
testing_ds=testing_dataset.shuffle(1024).batch(hyperparameters['batch_size']).prefetch(tf.data.AUTOTUNE)


In [ ]:
rescale = tf.keras.layers.Rescaling(1./255)
def data_preprocessing(batch):
  image = rescale(batch[0])
  label = tf.one_hot(batch[1],depth=10)
  return image,label

In [ ]:

class Graph_CN_Layer(tf.keras.layers.Layer):
  def __init__(self,input_channels,op_dimensions,neighbour_size=3):
    super(Graph_CN_Layer,self).__init__()
    self.op_dimensions = op_dimensions
    self.input_channels = input_channels
    self.a_matrix = tf.constant(np.ones((neighbour_size,neighbour_size)),dtype=tf.float32)
    self.a_matrix = tf.reshape(self.a_matrix,(neighbour_size,neighbour_size,1,1))
    self.a_matrix = tf.tile(self.a_matrix,(1,1,self.input_channels,self.op_dimensions))
    self.d_matrix = tf.constant([[0.57735027,0, 0],
                                 [0,0.57735027, 0],
                                 [0, 0, 0.57735027]],dtype=tf.float32)
    self.d_matrix = tf.reshape(self.d_matrix,(neighbour_size,neighbour_size,1,1))
    self.d_matrix = tf.tile(self.d_matrix,(1,1,self.input_channels,self.op_dimensions))
  def build(self,input_shape):
    pass

  def call(self,inputs):
    temp = tf.nn.conv2d(inputs,self.d_matrix,strides=1,padding='SAME')
    temp = tf.nn.conv2d(temp,self.a_matrix,strides=1,padding='SAME')
    temp = tf.nn.conv2d(temp,self.d_matrix,strides=1,padding='SAME')
    return temp 

In [ ]:
length = 3
a_matrix = tf.ones((length,length)
a_matrix= tf.reshape(a_matrix,(3,3,1,1))
a_matrix = tf.tile(a_matrix,(1,1,64,64))
d_matrix = tf.constant([[0.57735027,0, 0],
                        [0,0.57735027, 0],
                        [0, 0, 0.57735027]],dtype=tf.float32)
d_matrix = tf.reshape(d_matrix,(3,3,1,1))
d_matrix = tf.tile(d_matrix,(1,1,64,64))

SyntaxError: ignored

In [ ]:
x = layers.Conv2D(512,kernel_size=8,strides=2,activation='relu')(image_input)
x = layers.Conv2D(256,kernel_size=5,strides=1,activation='relu')(x)
#x = tf.nn.conv2d(x,d_matrix,strides=1,padding='SAME')
#x = Graph_CN_Layer(64,64)(x)
#x = layers.Flatten()(x)
x = layers.Conv2D(128,kernel_size=3,activation='relu',)(x)
x = layers.Conv2D(64,kernel_size=3,activation='relu',use_bias=True)(x)
x = layers.Conv2D(32,kernel_size=3,activation='relu',use_bias=True)(x)
x= layers.Flatten()(x)
output = layers.Dense(10,activation='softmax')(x)

model = keras.Model(inputs=image_input,outputs=output,name='Graph_Model')


In [ ]:

base_model = tf.keras.applications.ResNet50(include_top=False,input_shape=(64,64,3),classes=10)
for layer in base_model.layers:
  layers.trainable=False

In [ ]:
x = base_model.output
x = layers.Flatten()(x)
x = layers.Dense(1024,activation='relu')(x)
x = layers.Dense(10,activation='softmax')(x)
model = keras.Model(inputs=base_model.input,outputs=x,name='Graph_Model')

In [ ]:
print(base_model.summary())

In [ ]:
print(model.summary())

Model: "Graph_Model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1_conv[0][0]                 
________________________________________________________________________________________

In [ ]:
model.compile(optimizer='Adam',loss=tf.keras.losses.categorical_crossentropy
              ,metrics=[keras.metrics.CategoricalAccuracy(),keras.metrics.Accuracy()])

In [ ]:
for i in range(0,hyperparameters['epochs']):
  for j in range(0,len(training_dataset)% hyperparameters['batch_size']):
    image,label = data_preprocessing(next(iter(training_ds)))
    model.fit(image,label,verbose=1,epochs=1)

1/1 [==============================] - 0s 145ms/step - loss: 0.1273 - categorical_accuracy: 0.9375 - accuracy: 0.0063
